In [1]:
import numpy

import ivvv

In [2]:
image = numpy.random.random((10, 256, 256, 3))

ivvv.volshow(image, size=(256, 256))

ValueError: Images of type float must be between -1 and 1.

In [25]:
import math

def atlas_dimensions(aics_image, max_edge=2048, channel_names=None, physical_pixel_size=(1.0, 1.0, 1.0)):
    tile_width, tile_height, stack_height = aics_image.shape[1], aics_image.shape[2], aics_image.shape[0]    
    # maintain aspect ratio of images
    # initialize atlas with one row of all slices
    atlas_width = tile_width * stack_height
    atlas_height = tile_height
    ratio = float(atlas_width) / float(atlas_height)
    
    # these next steps attempt to optimize the atlas into a square shape
    # TODO - there must be a way to do this with a single calculation
    for r in range(2, stack_height):
        new_rows = math.ceil(float(stack_height) / r)
        adjusted_width = int(tile_width * new_rows)
        adjusted_height = int(tile_height * r)
        new_ratio = float(max(adjusted_width, adjusted_height)) / float(min(adjusted_width, adjusted_height))
        if new_ratio < ratio:
            ratio = new_ratio
            atlas_width = adjusted_width
            atlas_height = adjusted_height
        else:
            # we've found the rows and columns that make this the most square image
            break
    
    cols = int(atlas_width // tile_width)
    rows = int(atlas_height // tile_height)
    
    if max_edge < atlas_width or max_edge < atlas_height:
        tile_width = m.floor(max_edge/cols)
        tile_height = m.floor(max_edge/rows)
        atlas_width = tile_width * cols
        atlas_height = tile_height * rows
    dims = {}
    dims["tile_width"] = int(tile_width)
    dims["tile_height"] = int(tile_height)
    dims["rows"] = int(rows)
    dims["cols"] = int(cols)
    dims["atlas_width"] = int(atlas_width)
    dims["atlas_height"] = int(atlas_height)
    dims["width"] = aics_image.shape[1]
    dims["height"] = aics_image.shape[2]
    dims["channels"] = aics_image.shape[3]
    dims["tiles"] = aics_image.shape[0]
    
    if channel_names is not None:
        dims["channel_names"] = channel_names
    else:
        dims["channel_names"] = ['CH_'+str(i) for i in range(aics_image.shape[3])]
    
    if physical_pixel_size is not None:
        dims["pixel_size_x"] = physical_pixel_size[0]
        dims["pixel_size_y"] = physical_pixel_size[1]
        dims["pixel_size_z"] = physical_pixel_size[2]
    else:
        dims["pixel_size_x"] = 1
        dims["pixel_size_y"] = 1
        dims["pixel_size_z"] = 1
    
    return dims

In [27]:
atlas_dimensions(image)

{'atlas_height': 768,
 'atlas_width': 1024,
 'channel_names': ['CH_0', 'CH_1', 'CH_2'],
 'channels': 3,
 'cols': 4,
 'height': 256,
 'pixel_size_x': 1.0,
 'pixel_size_y': 1.0,
 'pixel_size_z': 1.0,
 'rows': 3,
 'tile_height': 256,
 'tile_width': 256,
 'tiles': 10,
 'width': 256}

In [5]:
image.shape

(10, 256, 256, 3)